In [20]:
import scipy as sp
import numpy as np
import time

try:
    from localgraphclustering import *
except:
    # when the package is not installed, import the local version instead. 
    # the notebook must be placed in the original "notebooks/" folder
    sys.path.append("../")
    from localgraphclustering import * 

import time

import networkx as nx

import random

import statistics as stat_

## Load data

In [21]:
g = GraphLocal('./datasets/ppi_mips.graphml','graphml',' ')

## TEMP

In [22]:
G = nx.read_graphml('./datasets/ppi_mips.graphml')

In [23]:
# groups = np.loadtxt('./datasets/ppi_mips.class', dtype = 'float')
groups = np.loadtxt('./datasets/ppi_mips_ground_truth.csv', dtype = 'str')

groups_by_id = dict()

for node in groups:
    groups_by_id[node[0]] = node[1]

ids_clusters = set()

for node in groups:
    ids_clusters.add(node[1])
    
ids_clusters = list(ids_clusters)

ground_truth_clusters_by_id = dict()

for node in groups:
    ground_truth_clusters_by_id[node[1]] =  []
    
for node in groups:
    ground_truth_clusters_by_id[node[1]].append(node[0])
    
ground_truth_clusters_by_number = dict()

for node in groups:
    ground_truth_clusters_by_number[node[1]] =  []

counter = 0
for node in G.node:
    
    if node == '1.0':
        counter += 1
        continue
    
    what_group = groups_by_id[node]
    ground_truth_clusters_by_number[what_group].append(counter)
    counter += 1
    
all_clusters = []
    
counter = 0
for cluster_id in ground_truth_clusters_by_number:
    
    cluster = ground_truth_clusters_by_number[cluster_id]
    
    if len(cluster) == 1 or len(cluster) == 0:
        counter += 1
        continue
    
#     eig, lambda_ = fiedler_local(g, cluster)
#     lambda_ = np.real(lambda_)
#     gap = lambda_/g.compute_conductance(cluster)
    cond = g.compute_conductance(cluster)
    counter += 1
    
    if cond <= 0.57 and len(cluster) >= 10:
        print("Id: ", cluster_id)
        print("Cluster: ", counter, " conductance: ", cond, "Size: ", len(cluster), " Volume: ", np.sum(g.d[cluster]))
        all_clusters.append(cluster)

Id:  Actin-associated-proteins:1.0
Cluster:  3  conductance:  0.36551724137931035 Size:  24  Volume:  870.0
Id:  Anaphase-promoting-complex:1.0
Cluster:  7  conductance:  0.3333333333333333 Size:  11  Volume:  165.0
Id:  Cdc28p-complexes:1.0
Cluster:  25  conductance:  0.3333333333333333 Size:  10  Volume:  135.0
Id:  Coat-complexes:1.0
Cluster:  31  conductance:  0.4940828402366864 Size:  19  Volume:  676.0
Id:  cytoplasmic-ribosomal-large-subunit:1.0
Cluster:  35  conductance:  0.3333333333333333 Size:  81  Volume:  9720.0
Id:  cytoplasmic-ribosomal-small-subunit:1.0
Cluster:  36  conductance:  0.3333333333333333 Size:  57  Volume:  4788.0
Id:  F0-F1-ATP-synthase:1.0
Cluster:  63  conductance:  0.3333333333333333 Size:  15  Volume:  315.0
Id:  H+-transporting-ATPase-vacuolar:1.0
Cluster:  77  conductance:  0.3333333333333333 Size:  15  Volume:  315.0
Id:  mitochondrial-ribosomal-large-subunit:1.0
Cluster:  92  conductance:  0.3333333333333333 Size:  32  Volume:  1488.0
Id:  mitochond

## Collect data for ACL (with rounding)

In [24]:
nodes = {}
external_best_cond_acl = {}
external_best_pre_cond_acl = {}
vol_best_cond_acl = {}
vol_best_pre_acl = {}
size_clust_best_cond_acl = {}
size_clust_best_pre_acl = {}
f1score_best_cond_acl = {}
f1score_best_pre_acl = {}
true_positives_best_cond_acl = {}
true_positives_best_pre_acl = {}
precision_best_cond_acl = {}
precision_best_pre_acl = {}
recall_best_cond_acl = {}
recall_best_pre_acl = {}
cuts_best_cond_acl = {}
cuts_best_pre_acl = {}
cuts_acl_ALL = {}

ct_outer = 0

start = time.time()

number_experiments = 0

for rr in all_clusters:
    
    how_many = int(len(rr))
    print(how_many)
    
    random.seed(4)
    
    nodes[ct_outer] = np.random.choice(rr, how_many, replace=False)
    
    eigv, lambda_val = fiedler_local(g, rr)
    lambda_val = np.real(lambda_val)
    
    step = (2*lambda_val - lambda_val/2)/4
    
    a_list = np.arange(lambda_val/2,2*lambda_val,step)
    
    ct = 0
    for node in nodes[ct_outer]:
        ref_node = [node]
        
        max_precision = -1
        min_conduct = 100
        
        ct_inner = 0
        for a in a_list:
            
            if ct_outer <= 1:
                rho = 0.15/np.sum(g.d[rr])
            else:
                rho = 0.2/np.sum(g.d[rr])
            
            output_pr_clustering = approximate_PageRank(g,ref_node,method = "acl", rho=rho, alpha=a, cpp = True, normalize=True,normalized_objective=True)
            number_experiments += 1
            
            output_pr_sc = sweep_cut(g,output_pr_clustering,cpp=True)
            
            S = output_pr_sc[0]
            
            cuts_acl_ALL[ct_outer,node,ct_inner] = S
            
            size_clust_acl_ = len(S)
            
            cond_val_l1pr = g.compute_conductance(S)
            
            vol_ = sum(g.d[S])
            true_positives_acl_ = set(rr).intersection(S)
            if len(true_positives_acl_) == 0:
                true_positives_acl_ = set(ref_node)
                vol_ = g.d[ref_node][0,0]
            precision = sum(g.d[np.array(list(true_positives_acl_))])/vol_
            recall = sum(g.d[np.array(list(true_positives_acl_))])/sum(g.d[rr])
            f1_score_ = 2*(precision*recall)/(precision + recall)
            
            if f1_score_ >= max_precision:
                
                max_precision = f1_score_
                
                external_best_pre_cond_acl[ct_outer,node] = cond_val_l1pr
                vol_best_pre_acl[ct_outer,node] = vol_
                
                size_clust_best_pre_acl[ct_outer,node] = size_clust_acl_
                true_positives_best_pre_acl[ct_outer,node] = true_positives_acl_
                precision_best_pre_acl[ct_outer,node] = precision
                recall_best_pre_acl[ct_outer,node] = recall
                f1score_best_pre_acl[ct_outer,node] = f1_score_
                
                cuts_best_pre_acl[ct_outer,node] = S
        
            if cond_val_l1pr <= min_conduct:
                
                min_conduct = cond_val_l1pr
                
                external_best_cond_acl[ct_outer,node] = cond_val_l1pr
                vol_best_cond_acl[ct_outer,node] = vol_
                
                size_clust_best_cond_acl[ct_outer,node] = size_clust_acl_
                true_positives_best_cond_acl[ct_outer,node] = true_positives_acl_
                precision_best_cond_acl[ct_outer,node] = precision
                recall_best_cond_acl[ct_outer,node] = recall
                f1score_best_cond_acl[ct_outer,node] = f1_score_
                
                cuts_best_cond_acl[ct_outer,node] = S

        print('outer:', ct_outer, 'number of node: ',node, ' completed: ', ct/how_many, ' degree: ', g.d[node])
        print('conductance: ', external_best_cond_acl[ct_outer,node], 'f1score: ', f1score_best_pre_acl[ct_outer,node], 'precision: ', precision_best_pre_acl[ct_outer,node], 'recall: ', recall_best_pre_acl[ct_outer,node])
        ct += 1
    end = time.time()
    print(" ")
    print("Outer: ", ct_outer," Elapsed time ACL with rounding: ", end - start)
    print("Outer: ", ct_outer," Number of experiments: ", number_experiments)
    print(" ")
    ct_outer += 1

24
outer: 0 number of node:  842  completed:  0.0  degree:  42.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  854  completed:  0.041666666666666664  degree:  29.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  839  completed:  0.08333333333333333  degree:  45.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  846  completed:  0.125  degree:  38.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  843  completed:  0.16666666666666666  degree:  41.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  844  completed:  0.20833333333333334  degree:  40.0
conductance:  0.333333333333

outer: 3 number of node:  597  completed:  0.2631578947368421  degree:  41.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
outer: 3 number of node:  598  completed:  0.3157894736842105  degree:  40.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
outer: 3 number of node:  599  completed:  0.3684210526315789  degree:  39.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
outer: 3 number of node:  609  completed:  0.42105263157894735  degree:  28.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
outer: 3 number of node:  592  completed:  0.47368421052631576  degree:  46.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
outer: 3 number of node:  604  completed:  0.5263157894736842  degree:  33.0
conduc

outer: 4 number of node:  135  completed:  0.4444444444444444  degree:  87.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  112  completed:  0.4567901234567901  degree:  110.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  129  completed:  0.4691358024691358  degree:  93.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  142  completed:  0.48148148148148145  degree:  80.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  76  completed:  0.49382716049382713  degree:  146.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  95  completed:  0.5061728395061729  degree:  127.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  121  completed:  0.5185185185185185  degree:  101.0
c

outer: 5 number of node:  35  completed:  0.15789473684210525  degree:  78.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  1  completed:  0.17543859649122806  degree:  111.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  56  completed:  0.19298245614035087  degree:  57.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  38  completed:  0.21052631578947367  degree:  75.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  48  completed:  0.22807017543859648  degree:  65.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  6  completed:  0.24561403508771928  degree:  107.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  32  completed:  0.2631578947368421  degree:  81.0
conduc

outer: 6 number of node:  661  completed:  0.4  degree:  17.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 6 number of node:  656  completed:  0.4666666666666667  degree:  22.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 6 number of node:  652  completed:  0.5333333333333333  degree:  26.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 6 number of node:  655  completed:  0.6  degree:  23.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 6 number of node:  660  completed:  0.6666666666666666  degree:  18.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 6 number of node:  650  completed:  0.7333333333333333  degree:  28.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 6 number of node:  651  completed:  0.8  degree:  27.0
conductance:  0.3333333333333333 f1score:  1.0 pre

outer: 9 number of node:  871  completed:  0.07142857142857142  degree:  21.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 9 number of node:  876  completed:  0.14285714285714285  degree:  16.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 9 number of node:  877  completed:  0.21428571428571427  degree:  15.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 9 number of node:  873  completed:  0.2857142857142857  degree:  19.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 9 number of node:  874  completed:  0.35714285714285715  degree:  18.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 9 number of node:  870  completed:  0.42857142857142855  degree:  22.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 9 number of node:  868  completed:  0.5  degree:  24.0
conductance:  0

outer: 11 number of node:  342  completed:  0.6363636363636364  degree:  71.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  360  completed:  0.6666666666666666  degree:  53.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  345  completed:  0.696969696969697  degree:  68.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  375  completed:  0.7272727272727273  degree:  38.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  354  completed:  0.7575757575757576  degree:  59.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  368  completed:  0.7878787878787878  degree:  45.0
con

outer: 13 number of node:  402  completed:  0.6206896551724138  degree:  38.0
conductance:  0.34053081421502474 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 13 number of node:  392  completed:  0.6551724137931034  degree:  50.0
conductance:  0.34053081421502474 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 13 number of node:  386  completed:  0.6896551724137931  degree:  57.0
conductance:  0.34053081421502474 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 13 number of node:  381  completed:  0.7241379310344828  degree:  64.0
conductance:  0.34053081421502474 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 13 number of node:  406  completed:  0.7586206896551724  degree:  34.0
conductance:  0.34053081421502474 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 13 number of node:  394  completed:  0.7931034482758621  degree:  48.0
conductance:  0.34053081421502474 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 13 number of node:  390  completed:  0.8275862068965517  degree

outer: 17 number of node:  545  completed:  0.0  degree:  26.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 17 number of node:  551  completed:  0.05555555555555555  degree:  20.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 17 number of node:  540  completed:  0.1111111111111111  degree:  31.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 17 number of node:  538  completed:  0.16666666666666666  degree:  33.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 17 number of node:  548  completed:  0.2222222222222222  degree:  23.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 17 number of node:  537  completed:  0.2777777777777778  degree:  34.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 17 number of node:  542  completed:  0.3333333333333333  degree:  29.0
conductance

## Performance of ACL (with rounding).

In [25]:
all_data = []
xlabels_ = []

print('Results for ACL with rounding')
sum_precision = 0
sum_recall = 0
sum_f1 = 0
sum_conductance = 0

info_ref_nodes = all_clusters
l_info_ref_nodes = len(info_ref_nodes)

for i in range(l_info_ref_nodes):
    temp_pre = []
    temp_rec = []
    temp_f1 = []
    temp_conductance = []
    
    for j in all_clusters[i]:
        temp_pre.append(precision_best_cond_acl[i,j])
        temp_rec.append(recall_best_cond_acl[i,j])
        temp_f1.append(f1score_best_cond_acl[i,j])
        temp_conductance.append(external_best_cond_acl[i,j])
    
    print('Feature:', i,'Precision', stat_.mean(temp_pre), 'Recall', stat_.mean(temp_rec), 'F1', stat_.mean(temp_f1), 'Cond.', stat_.mean(temp_conductance))



Results for ACL with rounding
Feature: 0 Precision 0.9206349206349206 Recall 1.0 F1 0.9586776859504132 Cond. 0.3333333333333333
Feature: 1 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 2 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 3 Precision 0.7229946524064171 Recall 1.0 F1 0.8392302917442582 Cond. 0.3582887700534759
Feature: 4 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 5 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 6 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 7 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 8 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 9 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 10 Precision 0.6862979215865966 Recall 1.0 F1 0.809935653577124 Cond. 0.4173334273260254
Feature: 11 Precision 0.8824087245139877 Recall 1.0 F1 0.9375314861460956 Cond. 0.3333333333333333
Feature: 12 Precision 0.709826589

## Collect data for l1-reg. PR (with rounding)

In [26]:
nodes = {}
external_best_cond_acl = {}
external_best_pre_cond_acl = {}
vol_best_cond_acl = {}
vol_best_pre_acl = {}
size_clust_best_cond_acl = {}
size_clust_best_pre_acl = {}
f1score_best_cond_acl = {}
f1score_best_pre_acl = {}
true_positives_best_cond_acl = {}
true_positives_best_pre_acl = {}
precision_best_cond_acl = {}
precision_best_pre_acl = {}
recall_best_cond_acl = {}
recall_best_pre_acl = {}
cuts_best_cond_acl = {}
cuts_best_pre_acl = {}
cuts_acl_ALL = {}

ct_outer = 0

start = time.time()

number_experiments = 0

for rr in all_clusters:
    
    how_many = int(len(rr))
    print(how_many)
    
    random.seed(4)
    
    nodes[ct_outer] = np.random.choice(rr, how_many, replace=False)
    
    eigv, lambda_val = fiedler_local(g, rr)
    lambda_val = np.real(lambda_val)
    
    step = (2*lambda_val - lambda_val/2)/4
    
    a_list = np.arange(lambda_val/2,2*lambda_val,step)
    
    ct = 0
    for node in nodes[ct_outer]:
        ref_node = [node]
        
        max_precision = -1
        min_conduct = 100
        
        ct_inner = 0
        for a in a_list:
            
            if ct_outer <= 1:
                rho = 0.15/np.sum(g.d[rr])
            else:
                rho = 0.2/np.sum(g.d[rr])
            
            output_pr_clustering = approximate_PageRank(g,ref_node,method = "l1reg-rand", epsilon=1.0e-2, rho=rho, alpha=a, cpp = True, normalize=True,normalized_objective=True)
            number_experiments += 1
            
            output_pr_sc = sweep_cut(g,output_pr_clustering,cpp=True)
            
            S = output_pr_sc[0]
            
            cuts_acl_ALL[ct_outer,node,ct_inner] = S
            
            size_clust_acl_ = len(S)
            
            cond_val_l1pr = g.compute_conductance(S)
            
            vol_ = sum(g.d[S])
            true_positives_acl_ = set(rr).intersection(S)
            if len(true_positives_acl_) == 0:
                true_positives_acl_ = set(ref_node)
                vol_ = g.d[ref_node][0,0]
            precision = sum(g.d[np.array(list(true_positives_acl_))])/vol_
            recall = sum(g.d[np.array(list(true_positives_acl_))])/sum(g.d[rr])
            f1_score_ = 2*(precision*recall)/(precision + recall)
            
            if f1_score_ >= max_precision:
                
                max_precision = f1_score_
                
                external_best_pre_cond_acl[ct_outer,node] = cond_val_l1pr
                vol_best_pre_acl[ct_outer,node] = vol_
                
                size_clust_best_pre_acl[ct_outer,node] = size_clust_acl_
                true_positives_best_pre_acl[ct_outer,node] = true_positives_acl_
                precision_best_pre_acl[ct_outer,node] = precision
                recall_best_pre_acl[ct_outer,node] = recall
                f1score_best_pre_acl[ct_outer,node] = f1_score_
                
                cuts_best_pre_acl[ct_outer,node] = S
        
            if cond_val_l1pr <= min_conduct:
                
                min_conduct = cond_val_l1pr
                
                external_best_cond_acl[ct_outer,node] = cond_val_l1pr
                vol_best_cond_acl[ct_outer,node] = vol_
                
                size_clust_best_cond_acl[ct_outer,node] = size_clust_acl_
                true_positives_best_cond_acl[ct_outer,node] = true_positives_acl_
                precision_best_cond_acl[ct_outer,node] = precision
                recall_best_cond_acl[ct_outer,node] = recall
                f1score_best_cond_acl[ct_outer,node] = f1_score_
                
                cuts_best_cond_acl[ct_outer,node] = S

        print('outer:', ct_outer, 'number of node: ',node, ' completed: ', ct/how_many, ' degree: ', g.d[node])
        print('conductance: ', external_best_cond_acl[ct_outer,node], 'f1score: ', f1score_best_pre_acl[ct_outer,node], 'precision: ', precision_best_pre_acl[ct_outer,node], 'recall: ', recall_best_pre_acl[ct_outer,node])
        ct += 1

    end = time.time()
    print(" ")
    print("Outer: ", ct_outer," Elapsed time l1-reg. with rounding: ", end - start)
    print("Outer: ", ct_outer," Number of experiments: ", number_experiments)
    print(" ")
    ct_outer += 1

24
outer: 0 number of node:  851  completed:  0.0  degree:  33.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  848  completed:  0.041666666666666664  degree:  36.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  855  completed:  0.08333333333333333  degree:  28.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  840  completed:  0.125  degree:  44.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  842  completed:  0.16666666666666666  degree:  42.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
outer: 0 number of node:  844  completed:  0.20833333333333334  degree:  40.0
conductance:  0.337016574585

outer: 3 number of node:  598  completed:  0.9473684210526315  degree:  40.0
conductance:  0.3603707995365006 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
 
Outer:  3  Elapsed time l1-reg. with rounding:  0.37760472297668457
Outer:  3  Number of experiments:  256
 
81
outer: 4 number of node:  131  completed:  0.0  degree:  91.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  86  completed:  0.012345679012345678  degree:  136.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  112  completed:  0.024691358024691357  degree:  110.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  130  completed:  0.037037037037037035  degree:  92.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  118  completed:  0.04938271604938271  degree:  104.0
conductance:  0.3333

outer: 4 number of node:  110  completed:  0.6666666666666666  degree:  112.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  136  completed:  0.6790123456790124  degree:  86.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  91  completed:  0.691358024691358  degree:  131.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  139  completed:  0.7037037037037037  degree:  83.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  120  completed:  0.7160493827160493  degree:  102.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  129  completed:  0.7283950617283951  degree:  93.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 4 number of node:  92  completed:  0.7407407407407407  degree:  130.0
cond

outer: 5 number of node:  19  completed:  0.6491228070175439  degree:  94.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  9  completed:  0.6666666666666666  degree:  104.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  20  completed:  0.6842105263157895  degree:  93.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  30  completed:  0.7017543859649122  degree:  83.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  35  completed:  0.7192982456140351  degree:  78.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  23  completed:  0.7368421052631579  degree:  90.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 5 number of node:  25  completed:  0.7543859649122807  degree:  88.0
conductance:

outer: 8 number of node:  470  completed:  0.15625  degree:  44.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 8 number of node:  462  completed:  0.1875  degree:  52.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 8 number of node:  471  completed:  0.21875  degree:  43.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 8 number of node:  477  completed:  0.25  degree:  37.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 8 number of node:  479  completed:  0.28125  degree:  35.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 8 number of node:  473  completed:  0.3125  degree:  41.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 8 number of node:  472  completed:  0.34375  degree:  42.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 8 nu

outer: 11 number of node:  359  completed:  0.030303030303030304  degree:  54.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  352  completed:  0.06060606060606061  degree:  61.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  376  completed:  0.09090909090909091  degree:  37.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  356  completed:  0.12121212121212122  degree:  57.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  344  completed:  0.15151515151515152  degree:  69.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
outer: 11 number of node:  339  completed:  0.18181818181818182  degree:  

outer: 12 number of node:  570  completed:  0.9444444444444444  degree:  31.0
conductance:  0.36184971098265895 f1score:  0.8302907369844489 precision:  0.7098265895953757 recall:  1.0
 
Outer:  12  Elapsed time l1-reg. with rounding:  3.2976949214935303
Outer:  12  Number of experiments:  1364
 
29
outer: 13 number of node:  399  completed:  0.0  degree:  43.0
conductance:  0.3808532778355879 f1score:  0.8723965972425932 precision:  0.7736732570239334 recall:  1.0
outer: 13 number of node:  403  completed:  0.034482758620689655  degree:  37.0
conductance:  0.3808532778355879 f1score:  0.8723965972425932 precision:  0.7736732570239334 recall:  1.0
outer: 13 number of node:  392  completed:  0.06896551724137931  degree:  50.0
conductance:  0.3808532778355879 f1score:  0.9408415058525783 precision:  0.8882915173237754 recall:  1.0
outer: 13 number of node:  404  completed:  0.10344827586206896  degree:  36.0
conductance:  0.3808532778355879 f1score:  0.8723965972425932 precision:  0.7736

outer: 15 number of node:  279  completed:  0.1  degree:  17.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 15 number of node:  280  completed:  0.2  degree:  16.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 15 number of node:  284  completed:  0.3  degree:  12.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 15 number of node:  285  completed:  0.4  degree:  11.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 15 number of node:  286  completed:  0.5  degree:  10.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 15 number of node:  283  completed:  0.6  degree:  13.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 15 number of node:  281  completed:  0.7  degree:  15.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
outer: 15 number of node:  

## Performance of l1-reg. PR (with rounding).

In [27]:
all_data = []
xlabels_ = []

print('Results for l1-reg with rounding')
sum_precision = 0
sum_recall = 0
sum_f1 = 0
sum_conductance = 0

info_ref_nodes = all_clusters
l_info_ref_nodes = len(info_ref_nodes)

for i in range(l_info_ref_nodes):
    temp_pre = []
    temp_rec = []
    temp_f1 = []
    temp_conductance = []
    
    for j in all_clusters[i]:
        temp_pre.append(precision_best_cond_acl[i,j])
        temp_rec.append(recall_best_cond_acl[i,j])
        temp_f1.append(f1score_best_cond_acl[i,j])
        temp_conductance.append(external_best_cond_acl[i,j])
    
    print('Feature:', i,'Precision', stat_.mean(temp_pre), 'Recall', stat_.mean(temp_rec), 'F1', stat_.mean(temp_f1), 'Cond.', stat_.mean(temp_conductance))



Results for l1-reg with rounding
Feature: 0 Precision 0.9613259668508287 Recall 1.0 F1 0.9802816901408451 Cond. 0.3370165745856354
Feature: 1 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 2 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 3 Precision 0.7483922812279147 Recall 1.0 F1 0.8557617594794483 Cond. 0.3591654140463284
Feature: 4 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 5 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 6 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 7 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 8 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 9 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 10 Precision 0.7123989836078486 Recall 0.8938315539739027 F1 0.7854550692274475 Cond. 0.46902709534149334
Feature: 11 Precision 0.8824087245139877 Recall 1.0 F1 0.9375314861460956 Cond. 0.3333333333333333
Feature: 12 P

## Function for seed set expansion using BFS

In [28]:
import queue
def seed_grow_bfs_steps(g,seeds,steps,vol_target,target_cluster):
    """
    grow the initial seed set through BFS until its size reaches 
    a given ratio of the total number of nodes.
    """
    Q = queue.Queue()
    visited = np.zeros(g._num_vertices)
    visited[seeds] = 1
    for s in seeds:
        Q.put(s)
    if isinstance(seeds,np.ndarray):
        seeds = seeds.tolist()
    else:
        seeds = list(seeds)
    for step in range(steps):
        for k in range(Q.qsize()):
            node = Q.get()
            si,ei = g.adjacency_matrix.indptr[node],g.adjacency_matrix.indptr[node+1]
            neighs = g.adjacency_matrix.indices[si:ei]
            for i in range(len(neighs)):
                if visited[neighs[i]] == 0:
                    visited[neighs[i]] = 1
                    seeds.append(neighs[i])
                    Q.put(neighs[i])
                    
                    vol_seeds = np.sum(g.d[seeds])
                    vol_target_intersection_input = np.sum(g.d[list(set(target_cluster).intersection(set(seeds)))])
                    sigma = vol_target_intersection_input/vol_target
                    
                    if sigma > 0.75 or vol_seeds > 0.25*g.vol_G:
                        break
                 
            vol_seeds = np.sum(g.d[seeds])
            vol_target_intersection_input = np.sum(g.d[list(set(target_cluster).intersection(set(seeds)))])
            sigma = vol_target_intersection_input/vol_target   
            
            if sigma > 0.75 or vol_seeds > 0.25*g.vol_G:
                break
               
        vol_seeds = np.sum(g.d[seeds])
        vol_target_intersection_input = np.sum(g.d[list(set(target_cluster).intersection(set(seeds)))])
        sigma = vol_target_intersection_input/vol_target
                
        if sigma > 0.75 or vol_seeds > 0.25*g.vol_G:
            break
    return seeds

## Collect data for seed set expansion + FlowImprove, try a lot of parameters

In [29]:
nodes = {}
external_best_cond_flBFS = {}
external_best_pre_cond_flBFS = {}
vol_best_cond_flBFS = {}
vol_best_pre_flBFS = {}
size_clust_best_cond_flBFS = {}
size_clust_best_pre_flBFS = {}
f1score_best_cond_flBFS = {}
f1score_best_pre_flBFS = {}
true_positives_best_cond_flBFS = {}
true_positives_best_pre_flBFS = {}
precision_best_cond_flBFS = {}
precision_best_pre_flBFS = {}
recall_best_cond_flBFS = {}
recall_best_pre_flBFS = {}
cuts_best_cond_flBFS = {}
cuts_best_pre_flBFS = {}
cuts_flBFS_ALL = {}

ct_outer = 0

start = time.time()

number_experiments = 0

for rr in all_clusters:
    
    how_many = int(len(rr))
    print(how_many)
    
    random.seed(4)
    
    nodes[ct_outer] = np.random.choice(rr, how_many, replace=False)
    
    n_step = 24
    
    vol_target = np.sum(g.d[rr])
    
    ct = 0
    for node in nodes[ct_outer]:
        ref_node = [node]
        
        max_precision = -1
        min_conduct = 100
                
        seeds = seed_grow_bfs_steps(g,[node],g._num_vertices,vol_target,rr)

        vol_input = np.sum(g.d[seeds])

        vol_graph_minus_input = np.sum(g.d[list(set(range(g._num_vertices)) - set(seeds))])

        vol_target_intersection_input = np.sum(g.d[list(set(rr).intersection(set(seeds)))])

        gamma = vol_input/vol_graph_minus_input
        
#         print("gamma: ", gamma)
        
        sigma = max(vol_target_intersection_input/vol_target,gamma)
        
#         print("sigma: ", sigma)

        delta = min(max((1/3)*(1.0/(1.0/sigma - 1)) - gamma,0),1)
        
#         print("(1.0/3.0)*(1.0/(1.0/sigma - 1)): ", (1.0/3.0)*(1.0/(1.0/sigma - 1)))

        print("DELTA: ", delta)
                
        S = flow_clustering(g,seeds,method="sl",delta=delta)[0]
        number_experiments += 1

        cuts_flBFS_ALL[ct_outer,node] = S

        size_clust_flBFS_ = len(S)

        cond_val_l1pr = g.compute_conductance(S)

        vol_ = sum(g.d[S])
        true_positives_flBFS_ = set(rr).intersection(S)
        if len(true_positives_flBFS_) == 0:
            true_positives_flBFS_ = set(ref_node)
            vol_ = g.d[ref_node][0]
        precision = sum(g.d[np.array(list(true_positives_flBFS_))])/vol_
        recall = sum(g.d[np.array(list(true_positives_flBFS_))])/sum(g.d[rr])
        f1_score_ = 2*(precision*recall)/(precision + recall)

        if f1_score_ >= max_precision:

            max_precision = f1_score_

            external_best_pre_cond_flBFS[ct_outer,node] = cond_val_l1pr
            vol_best_pre_flBFS[ct_outer,node] = vol_

            size_clust_best_pre_flBFS[ct_outer,node] = size_clust_flBFS_
            true_positives_best_pre_flBFS[ct_outer,node] = true_positives_flBFS_
            precision_best_pre_flBFS[ct_outer,node] = precision
            recall_best_pre_flBFS[ct_outer,node] = recall
            f1score_best_pre_flBFS[ct_outer,node] = f1_score_

            cuts_best_pre_flBFS[ct_outer,node] = S

        if cond_val_l1pr <= min_conduct:

            min_conduct = cond_val_l1pr

            external_best_cond_flBFS[ct_outer,node] = cond_val_l1pr
            vol_best_cond_flBFS[ct_outer,node] = vol_

            size_clust_best_cond_flBFS[ct_outer,node] = size_clust_flBFS_
            true_positives_best_cond_flBFS[ct_outer,node] = true_positives_flBFS_
            precision_best_cond_flBFS[ct_outer,node] = precision
            recall_best_cond_flBFS[ct_outer,node] = recall
            f1score_best_cond_flBFS[ct_outer,node] = f1_score_

            cuts_best_cond_flBFS[ct_outer,node] = S

        print('outer:', ct_outer, 'number of node: ',node, ' completed: ', ct/how_many, ' degree: ', g.d[node])
        print('conductance: ', external_best_cond_flBFS[ct_outer,node], 'f1score: ', f1score_best_pre_flBFS[ct_outer,node], 'precision: ', precision_best_pre_flBFS[ct_outer,node], 'recall: ', recall_best_pre_flBFS[ct_outer,node])
        ct += 1
    end = time.time()
    print(" ")
    print("Outer: ", ct_outer," Elapsed time BFS+SL: ", end - start)
    print("Outer: ", ct_outer," Number of experiments: ", number_experiments)
    print(" ")
    ct_outer += 1

24
DELTA:  0
outer: 0 number of node:  854  completed:  0.0  degree:  29.0
conductance:  0.9991864627399935 f1score:  0.0044066251329585165 precision:  0.002359258054018874 recall:  0.03333333333333333
DELTA:  0
outer: 0 number of node:  852  completed:  0.041666666666666664  degree:  32.0
conductance:  0.9986986579910533 f1score:  0.004861374857576908 precision:  0.0026026840178934524 recall:  0.0367816091954023
DELTA:  0
outer: 0 number of node:  855  completed:  0.08333333333333333  degree:  28.0
conductance:  0.9993491172402571 f1score:  0.004254995820986247 precision:  0.0022780896591001547 recall:  0.03218390804597701
DELTA:  0
outer: 0 number of node:  838  completed:  0.125  degree:  46.0
conductance:  0.9964253798033958 f1score:  0.006980802792321117 precision:  0.0037371029328133886 recall:  0.052873563218390804
DELTA:  0
outer: 0 number of node:  848  completed:  0.16666666666666666  degree:  36.0
conductance:  0.9980486218391739 f1score:  0.005467385526615537 precision:  0.

outer: 2 number of node:  330  completed:  0.4  degree:  10.0
conductance:  0.9998370406583558 f1score:  0.0016118633139909737 precision:  0.0008147967082212988 recall:  0.07407407407407407
DELTA:  0
outer: 2 number of node:  327  completed:  0.5  degree:  13.0
conductance:  0.999348321928967 f1score:  0.002094915800499557 precision:  0.0010589768654284783 recall:  0.0962962962962963
DELTA:  0
outer: 2 number of node:  329  completed:  0.6  degree:  11.0
conductance:  0.999674107870295 f1score:  0.0017729067612216936 precision:  0.0008962033566889359 recall:  0.08148148148148149
DELTA:  0
outer: 2 number of node:  323  completed:  0.7  degree:  17.0
conductance:  0.9986970684039088 f1score:  0.002738622633910592 precision:  0.0013843648208469055 recall:  0.1259259259259259
DELTA:  1
outer: 2 number of node:  331  completed:  0.8  degree:  9.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  0
outer: 2 number of node:  328  completed:  0.9  degree:  12

outer: 4 number of node:  102  completed:  0.18518518518518517  degree:  120.0
conductance:  0.9935395300008075 f1score:  0.010858254535583405 precision:  0.009690704998788663 recall:  0.012345679012345678
DELTA:  0
outer: 4 number of node:  76  completed:  0.19753086419753085  degree:  146.0
conductance:  0.9893625594326698 f1score:  0.013195354512178587 precision:  0.011765653960834878 recall:  0.015020576131687243
DELTA:  0
outer: 4 number of node:  135  completed:  0.20987654320987653  degree:  87.0
conductance:  0.9988663967611336 f1score:  0.007884005437245128 precision:  0.007044534412955466 recall:  0.008950617283950617
DELTA:  0
outer: 4 number of node:  128  completed:  0.2222222222222222  degree:  94.0
conductance:  0.997734077850611 f1score:  0.008515649771255152 precision:  0.007607024358663106 recall:  0.009670781893004115
DELTA:  0
outer: 4 number of node:  104  completed:  0.2345679012345679  degree:  118.0
conductance:  0.9938615620709151 f1score:  0.01067824985294783 

outer: 4 number of node:  130  completed:  0.6790123456790124  degree:  92.0
conductance:  0.9980574666127074 f1score:  0.008335220838052094 precision:  0.0074463779846216105 recall:  0.009465020576131687
DELTA:  0
outer: 4 number of node:  91  completed:  0.691358024691358  degree:  131.0
conductance:  0.9917702113926093 f1score:  0.011847698290675588 precision:  0.010569630466354688 recall:  0.013477366255144034
DELTA:  0
outer: 4 number of node:  92  completed:  0.7037037037037037  degree:  130.0
conductance:  0.9919309287501009 f1score:  0.011757789535567314 precision:  0.010489792624868878 recall:  0.013374485596707819
DELTA:  0
outer: 4 number of node:  93  completed:  0.7160493827160493  degree:  129.0
conductance:  0.9920916720464816 f1score:  0.011667872648335745 precision:  0.010409941897998709 recall:  0.013271604938271606
DELTA:  0
outer: 4 number of node:  115  completed:  0.7283950617283951  degree:  107.0
conductance:  0.9956345998383185 f1score:  0.009687641466727027 pr

outer: 5 number of node:  35  completed:  0.24561403508771928  degree:  78.0
conductance:  0.3579098672528959 f1score:  0.032059186189889025 precision:  1.0 recall:  0.016290726817042606
DELTA:  0
outer: 5 number of node:  46  completed:  0.2631578947368421  degree:  67.0
conductance:  0.3579098672528959 f1score:  0.02760041194644696 precision:  1.0 recall:  0.013993316624895571
DELTA:  0
outer: 5 number of node:  39  completed:  0.2807017543859649  degree:  74.0
conductance:  0.3579098672528959 f1score:  0.03044014808720691 precision:  1.0 recall:  0.015455304928989139
DELTA:  0
outer: 5 number of node:  1  completed:  0.2982456140350877  degree:  111.0
conductance:  0.9820599070959475 f1score:  0.025819150167882363 precision:  0.017860003203588017 recall:  0.04657477025898078
DELTA:  0
outer: 5 number of node:  3  completed:  0.3157894736842105  degree:  110.0
conductance:  0.3579098672528959 f1score:  0.044916292364230295 precision:  1.0 recall:  0.02297410192147034
DELTA:  0
outer:

...

outer: 10 number of node:  59  completed:  0.5  degree:  30.0
conductance:  0.9977222809729114 f1score:  0.004771751232702402 precision:  0.0024404132433092003 recall:  0.10676156583629894
DELTA:  0
outer: 10 number of node:  710  completed:  0.5833333333333334  degree:  23.0
conductance:  0.9985349177926095 f1score:  0.003660380361263627 precision:  0.0018720494872212273 recall:  0.08185053380782918
DELTA:  1
outer: 10 number of node:  61  completed:  0.6666666666666666  degree:  16.0
conductance:  0.43005181347150256 f1score:  0.8425787106446777 precision:  0.727979274611399 recall:  1.0
DELTA:  0
outer: 10 number of node:  711  completed:  0.75  degree:  22.0
conductance:  0.9986975986975987 f1score:  0.0035015120165526026 precision:  0.0017908017908017909 recall:  0.07829181494661921
DELTA:  0
outer: 10 number of node:  713  completed:  0.8333333333333334  degree:  18.0
conductance:  0.9993485872485954 f1score:  0.0028657857029135486 precision:  0.0014656786906603696 recall:  0.064

outer: 12 number of node:  563  completed:  0.05555555555555555  degree:  38.0
conductance:  0.9975611738883018 f1score:  0.005884630274874178 precision:  0.003089179741484432 recall:  0.06188925081433225
DELTA:  0
outer: 12 number of node:  569  completed:  0.1111111111111111  degree:  32.0
conductance:  0.9985359902399349 f1score:  0.00495778139282671 precision:  0.0026026840178934524 recall:  0.05211726384364821
DELTA:  0
outer: 12 number of node:  564  completed:  0.16666666666666666  degree:  37.0
conductance:  0.9977235772357723 f1score:  0.005730215270249342 precision:  0.0030081300813008132 recall:  0.06026058631921824
DELTA:  0
outer: 12 number of node:  570  completed:  0.2222222222222222  degree:  31.0
conductance:  0.998698552139255 f1score:  0.004803222807561202 precision:  0.0025215552301935903 recall:  0.050488599348534204
DELTA:  0
outer: 12 number of node:  573  completed:  0.2777777777777778  degree:  28.0
conductance:  0.9991863965503214 f1score:  0.00433940333204184

outer: 13 number of node:  382  completed:  0.8275862068965517  degree:  63.0
conductance:  0.9952944994320948 f1score:  0.009121841743285312 precision:  0.005111147168586727 recall:  0.04236718224613315
DELTA:  0
outer: 13 number of node:  395  completed:  0.8620689655172413  degree:  47.0
conductance:  0.9978878960194963 f1score:  0.006813075306225991 precision:  0.003818034118602762 recall:  0.03160726294552791
DELTA:  0
outer: 13 number of node:  381  completed:  0.896551724137931  degree:  64.0
conductance:  0.9951326356777804 f1score:  0.009265962067467786 precision:  0.005191855277034153 recall:  0.04303967720242098
DELTA:  0
outer: 13 number of node:  403  completed:  0.9310344827586207  degree:  37.0
conductance:  0.9995121951219512 f1score:  0.005367375063465583 precision:  0.0030081300813008132 recall:  0.02488231338264963
DELTA:  0
outer: 13 number of node:  385  completed:  0.9655172413793104  degree:  58.0
conductance:  0.9961042123204286 f1score:  0.008400926998841252 pr

outer: 16 number of node:  723  completed:  0.36363636363636365  degree:  14.0
conductance:  0.9993483750101816 f1score:  0.002250442051117184 precision:  0.0011403437321821292 recall:  0.08484848484848485
DELTA:  0
outer: 16 number of node:  717  completed:  0.45454545454545453  degree:  20.0
conductance:  0.9983717332899129 f1score:  0.003213367609254499 precision:  0.0016282667100871123 recall:  0.12121212121212122
DELTA:  1
outer: 16 number of node:  727  completed:  0.5454545454545454  degree:  10.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  0
outer: 16 number of node:  726  completed:  0.6363636363636364  degree:  11.0
conductance:  0.9998370539351474 f1score:  0.0017686309188841545 precision:  0.0008962033566889359 recall:  0.06666666666666667
DELTA:  0
outer: 16 number of node:  722  completed:  0.7272727272727273  degree:  15.0
conductance:  0.999185535103437 f1score:  0.002410994133247609 precision:  0.0012216973448444372 recall:  0.09

outer: 18 number of node:  491  completed:  0.9333333333333333  degree:  27.0
conductance:  0.9978844589096827 f1score:  0.004284014280047601 precision:  0.002196908055329536 recall:  0.08571428571428572
 
Outer:  18  Elapsed time BFS+SL:  47.10722279548645
Outer:  18  Number of experiments:  446
 


## Performance of BFS+FlowImp.

In [30]:
all_data = []
xlabels_ = []

print('Results for BFS+SL')
sum_precision = 0
sum_recall = 0
sum_f1 = 0
sum_conductance = 0

info_ref_nodes = all_clusters
l_info_ref_nodes = len(info_ref_nodes)

for i in range(l_info_ref_nodes):
    temp_pre = []
    temp_rec = []
    temp_f1 = []
    temp_conductance = []
    
    for j in all_clusters[i]:
        temp_pre.append(precision_best_cond_flBFS[i,j])
        temp_rec.append(recall_best_cond_flBFS[i,j])
        temp_f1.append(f1score_best_cond_flBFS[i,j])
        temp_conductance.append(external_best_cond_flBFS[i,j])

    print('Feature:', i,'Precision', stat_.mean(temp_pre), 'Recall', stat_.mean(temp_rec), 'F1', stat_.mean(temp_f1), 'Cond.', stat_.mean(temp_conductance))



Results for BFS+SL
Feature: 0 Precision 0.04292084923600127 Recall 0.08218390804597701 F1 0.046197704063043465 Cond. 0.9703843940597869
Feature: 1 Precision 0.0920566495716423 Recall 0.1763085399449036 F1 0.09317377642639872 Cond. 0.9385796071139778
Feature: 2 Precision 0.10102626959668703 Recall 0.19333333333333333 F1 0.10203021545956412 Cond. 0.9326003348027058
Feature: 3 Precision 0.04084210929419071 Recall 0.10339458112737465 F1 0.049458876092269115 Cond. 0.9643434889431524
Feature: 4 Precision 0.021952836080109214 Recall 0.024589747497840778 F1 0.023112177370761895 Cond. 0.9853161845250714
Feature: 5 Precision 0.9655389474808277 Recall 0.0356996291899339 F1 0.050907490168524853 Cond. 0.3750670287716839
Feature: 6 Precision 0.0683000611238342 Recall 0.13037037037037036 F1 0.06985178749988656 Cond. 0.954416108913791
Feature: 7 Precision 0.0683000611238342 Recall 0.13037037037037036 F1 0.06985178749988656 Cond. 0.954416108913791
Feature: 8 Precision 0.03494821107527765 Recall 0.06184

## Collect data for APPR+SL

In [31]:
nodes = {}
external_best_cond_apprSL = {}
external_best_pre_cond_apprSL = {}
vol_best_cond_apprSL = {}
vol_best_pre_apprSL = {}
size_clust_best_cond_apprSL = {}
size_clust_best_pre_apprSL = {}
f1score_best_cond_apprSL = {}
f1score_best_pre_apprSL = {}
true_positives_best_cond_apprSL = {}
true_positives_best_pre_apprSL = {}
precision_best_cond_apprSL = {}
precision_best_pre_apprSL = {}
recall_best_cond_apprSL = {}
recall_best_pre_apprSL = {}
cuts_best_cond_apprSL = {}
cuts_best_pre_apprSL = {}
cuts_apprSL_ALL = {}

ct_outer = 0

start = time.time()

number_experiments = 0

for rr in all_clusters:
    
    how_many = int(len(rr))
    print(how_many)
    
    random.seed(4)
    
    nodes[ct_outer] = np.random.choice(rr, how_many, replace=False)
    
    eigv, lambda_val = fiedler_local(g, rr)
    lambda_val = np.real(lambda_val)
    
    step = (2*lambda_val - lambda_val/2)/4
    
    a_list = np.arange(lambda_val/2,2*lambda_val,step)
    
    vol_target = np.sum(g.d[rr])
    
    ct = 0
    for node in nodes[ct_outer]:
        ref_node = [node]
        
        max_precision = -1
        min_conduct = 100
        
        ct_inner = 0
        for a in a_list:
            
            if ct_outer <= 1:
                rho = 0.15/np.sum(g.d[rr])
            else:
                rho = 0.2/np.sum(g.d[rr])
            
            output_pr_clustering = approximate_PageRank(g,ref_node,method = "acl", rho=rho, alpha=a, cpp = True, normalize=True,normalized_objective=True)
            number_experiments += 1
            
            output_pr_sc = sweep_cut(g,output_pr_clustering,cpp=True)
            
            S = output_pr_sc[0]
            
            vol_input = np.sum(g.d[S])

            vol_graph_minus_input = np.sum(g.d[list(set(range(g._num_vertices)) - set(S))])

            vol_target_intersection_input = np.sum(g.d[list(set(rr).intersection(set(S)))])

            gamma = vol_input/vol_graph_minus_input

    #         print("gamma: ", gamma)

            sigma = max(vol_target_intersection_input/vol_target,gamma)

    #         print("sigma: ", sigma)

            delta = min(max((1/3)*(1.0/(1.0/sigma - 1)) - gamma,0),1)

    #         print("(1.0/3.0)*(1.0/(1.0/sigma - 1)): ", (1.0/3.0)*(1.0/(1.0/sigma - 1)))

            print("DELTA: ", delta)

            S = flow_clustering(g,S,method="sl",delta=delta)[0]
            
            cuts_apprSL_ALL[ct_outer,node,ct_inner] = S
            
            size_clust_apprSL_ = len(S)
            
            cond_val_l1pr = g.compute_conductance(S)
            
            vol_ = sum(g.d[S])
            true_positives_apprSL_ = set(rr).intersection(S)
            if len(true_positives_apprSL_) == 0:
                true_positives_apprSL_ = set(ref_node)
                vol_ = g.d[ref_node][0]
            precision = sum(g.d[np.array(list(true_positives_apprSL_))])/vol_
            recall = sum(g.d[np.array(list(true_positives_apprSL_))])/sum(g.d[rr])
            f1_score_ = 2*(precision*recall)/(precision + recall)
            
            if f1_score_ >= max_precision:
                
                max_precision = f1_score_
                
                external_best_pre_cond_apprSL[ct_outer,node] = cond_val_l1pr
                vol_best_pre_apprSL[ct_outer,node] = vol_
                
                size_clust_best_pre_apprSL[ct_outer,node] = size_clust_apprSL_
                true_positives_best_pre_apprSL[ct_outer,node] = true_positives_apprSL_
                precision_best_pre_apprSL[ct_outer,node] = precision
                recall_best_pre_apprSL[ct_outer,node] = recall
                f1score_best_pre_apprSL[ct_outer,node] = f1_score_
                
                cuts_best_pre_apprSL[ct_outer,node] = S
        
            if cond_val_l1pr <= min_conduct:
                
                min_conduct = cond_val_l1pr
                
                external_best_cond_apprSL[ct_outer,node] = cond_val_l1pr
                vol_best_cond_apprSL[ct_outer,node] = vol_
                
                size_clust_best_cond_apprSL[ct_outer,node] = size_clust_apprSL_
                true_positives_best_cond_apprSL[ct_outer,node] = true_positives_apprSL_
                precision_best_cond_apprSL[ct_outer,node] = precision
                recall_best_cond_apprSL[ct_outer,node] = recall
                f1score_best_cond_apprSL[ct_outer,node] = f1_score_
                
                cuts_best_cond_apprSL[ct_outer,node] = S

        print('outer:', ct_outer, 'number of node: ',node, ' completed: ', ct/how_many, ' degree: ', g.d[node])
        print('conductance: ', external_best_cond_apprSL[ct_outer,node], 'f1score: ', f1score_best_pre_apprSL[ct_outer,node], 'precision: ', precision_best_pre_apprSL[ct_outer,node], 'recall: ', recall_best_pre_apprSL[ct_outer,node])
        ct += 1
    end = time.time()
    print(" ")
    print("Outer: ", ct_outer," Elapsed time APPR+SL with rounding: ", end - start)
    print("Outer: ", ct_outer," Number of experiments: ", number_experiments)
    print(" ")
    ct_outer += 1

24
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  858  completed:  0.0  degree:  25.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  856  completed:  0.041666666666666664  degree:  27.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1


/usr/local/Cellar/ipython/7.1.1/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning:

divide by zero encountered in double_scalars



DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  837  completed:  0.08333333333333333  degree:  47.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  842  completed:  0.125  degree:  42.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  844  completed:  0.16666666666666666  degree:  40.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  843  completed:  0.20833333333333334  degree:  41.0
conductance:  0.3333333333333333 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  853  completed:  0.25  degree:  31.0
conductance:  0.333333333

DELTA:  1
DELTA:  1
outer: 2 number of node:  326  completed:  0.6  degree:  14.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 2 number of node:  328  completed:  0.7  degree:  12.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 2 number of node:  330  completed:  0.8  degree:  10.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 2 number of node:  324  completed:  0.9  degree:  16.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
 
Outer:  2  Elapsed time APPR+SL with rounding:  3.850748300552368
Outer:  2  Number of experiments:  180
 
19
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.020685762813650865
outer: 3 number of node:  597  completed:  0.0  degree:  41.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0

DELTA:  1
DELTA:  1
DELTA:  1
outer: 4 number of node:  79  completed:  0.1728395061728395  degree:  143.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 4 number of node:  67  completed:  0.18518518518518517  degree:  155.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 4 number of node:  75  completed:  0.19753086419753085  degree:  147.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 4 number of node:  128  completed:  0.20987654320987653  degree:  94.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 4 number of node:  102  completed:  0.2222222222222222  degree:  120.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
o

...

DELTA:  1
outer: 8 number of node:  454  completed:  0.65625  degree:  60.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 8 number of node:  477  completed:  0.6875  degree:  37.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 8 number of node:  471  completed:  0.71875  degree:  43.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 8 number of node:  453  completed:  0.75  degree:  61.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 8 number of node:  473  completed:  0.78125  degree:  41.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 8 number of node:  479  completed:  0.8125  degree:  35.0
conductance:  0.333

DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 11 number of node:  339  completed:  0.09090909090909091  degree:  74.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 11 number of node:  344  completed:  0.12121212121212122  degree:  69.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 11 number of node:  356  completed:  0.15151515151515152  degree:  57.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 11 number of node:  349  completed:  0.18181818181818182  degree:  64.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 11 number of node:  341  completed:  0.21212121212121

DELTA:  1
DELTA:  1
DELTA:  0.020620056829536076
outer: 12 number of node:  564  completed:  0.3888888888888889  degree:  37.0
conductance:  0.36184971098265895 f1score:  0.8727789623312011 precision:  0.7742749054224464 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.01420889818008548
outer: 12 number of node:  575  completed:  0.4444444444444444  degree:  26.0
conductance:  0.36184971098265895 f1score:  0.8727789623312011 precision:  0.7742749054224464 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.01825966600677759
outer: 12 number of node:  568  completed:  0.5  degree:  33.0
conductance:  0.36184971098265895 f1score:  0.8727789623312011 precision:  0.7742749054224464 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.017674811178830154
outer: 12 number of node:  569  completed:  0.5555555555555556  degree:  32.0
conductance:  0.36184971098265895 f1score:  0.8727789623312011 precision:  0.7742749054224464 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.01884660713

DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 13 number of node:  391  completed:  0.8620689655172413  degree:  51.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.6689158794421952 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 13 number of node:  396  completed:  0.896551724137931  degree:  46.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.6689158794421952 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 13 number of node:  380  completed:  0.9310344827586207  degree:  65.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.6689158794421952 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 13 number of node:  406  completed:  0.9655172413793104  degree:  34.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.6689158794421952 recall:  1.0
 
Outer:  13  Elapsed time APPR+SL with rounding:  49.48203110694885
Outer:  13  Number of experiment

DELTA:  1
DELTA:  1
outer: 16 number of node:  718  completed:  0.2727272727272727  degree:  19.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 16 number of node:  717  completed:  0.36363636363636365  degree:  20.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 16 number of node:  719  completed:  0.45454545454545453  degree:  18.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 16 number of node:  726  completed:  0.5454545454545454  degree:  11.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 16 number of node:  727  completed:  0.6363636363636364  degree:  10.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 1

## Performance of APPR+SL

In [32]:
all_data = []
xlabels_ = []

print('Results for APPR+SL')
sum_precision = 0
sum_recall = 0
sum_f1 = 0
sum_conductance = 0

info_ref_nodes = all_clusters
l_info_ref_nodes = len(info_ref_nodes)

for i in range(l_info_ref_nodes):
    temp_pre = []
    temp_rec = []
    temp_f1 = []
    temp_conductance = []
    
    for j in all_clusters[i]:
        temp_pre.append(precision_best_cond_apprSL[i,j])
        temp_rec.append(recall_best_cond_apprSL[i,j])
        temp_f1.append(f1score_best_cond_apprSL[i,j])
        temp_conductance.append(external_best_cond_apprSL[i,j])

    print('Feature:', i,'Precision', stat_.mean(temp_pre), 'Recall', stat_.mean(temp_rec), 'F1', stat_.mean(temp_f1), 'Cond.', stat_.mean(temp_conductance))



Results for APPR+SL
Feature: 0 Precision 0.9206349206349206 Recall 1.0 F1 0.9586776859504132 Cond. 0.3333333333333333
Feature: 1 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 2 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 3 Precision 0.7229946524064171 Recall 1.0 F1 0.8392302917442582 Cond. 0.3582887700534759
Feature: 4 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 5 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 6 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 7 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 8 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 9 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 10 Precision 0.6795979847624363 Recall 1.0 F1 0.8035126726862769 Cond. 0.4151397962504067
Feature: 11 Precision 0.8824087245139877 Recall 1.0 F1 0.9375314861460956 Cond. 0.3333333333333333
Feature: 12 Precision 0.7098265895953757 R

## Collect data for L1+SL

In [33]:
nodes = {}
external_best_cond_l1SL = {}
external_best_pre_cond_l1SL = {}
vol_best_cond_l1SL = {}
vol_best_pre_l1SL = {}
size_clust_best_cond_l1SL = {}
size_clust_best_pre_l1SL = {}
f1score_best_cond_l1SL = {}
f1score_best_pre_l1SL = {}
true_positives_best_cond_l1SL = {}
true_positives_best_pre_l1SL = {}
precision_best_cond_l1SL = {}
precision_best_pre_l1SL = {}
recall_best_cond_l1SL = {}
recall_best_pre_l1SL = {}
cuts_best_cond_l1SL = {}
cuts_best_pre_l1SL = {}
cuts_l1SL_ALL = {}

ct_outer = 0

start = time.time()

number_experiments = 0

for rr in all_clusters:
    
    how_many = int(len(rr))
    print(how_many)
    
    random.seed(4)
    
    nodes[ct_outer] = np.random.choice(rr, how_many, replace=False)
    
    eigv, lambda_val = fiedler_local(g, rr)
    lambda_val = np.real(lambda_val)
    
    step = (2*lambda_val - lambda_val/2)/4
    
    a_list = np.arange(lambda_val/2,2*lambda_val,step)
    
    vol_target = np.sum(g.d[rr])
    
    ct = 0
    for node in nodes[ct_outer]:
        ref_node = [node]
        
        max_precision = -1
        min_conduct = 100
        
        ct_inner = 0
        for a in a_list:
            
            if ct_outer <= 1:
                rho = 0.15/np.sum(g.d[rr])
            else:
                rho = 0.2/np.sum(g.d[rr])
            
            output_pr_clustering = approximate_PageRank(g,ref_node,method = "l1reg-rand", epsilon=1.0e-2, rho=rho, alpha=a, cpp = True, normalize=True,normalized_objective=True)
            number_experiments += 1
            
            output_pr_sc = sweep_cut(g,output_pr_clustering,cpp=True)
            
            S = output_pr_sc[0]
            
            vol_input = np.sum(g.d[S])

            vol_graph_minus_input = np.sum(g.d[list(set(range(g._num_vertices)) - set(S))])

            vol_target_intersection_input = np.sum(g.d[list(set(rr).intersection(set(S)))])

            gamma = vol_input/vol_graph_minus_input

    #         print("gamma: ", gamma)

            sigma = max(vol_target_intersection_input/vol_target,gamma)

    #         print("sigma: ", sigma)

            delta = min(max((1/3)*(1.0/(1.0/sigma - 1)) - gamma,0),1)

    #         print("(1.0/3.0)*(1.0/(1.0/sigma - 1)): ", (1.0/3.0)*(1.0/(1.0/sigma - 1)))

            print("DELTA: ", delta)

            S = flow_clustering(g,S,method="sl",delta=delta)[0]
            
            cuts_l1SL_ALL[ct_outer,node,ct_inner] = S
            
            size_clust_l1SL_ = len(S)
            
            cond_val_l1pr = g.compute_conductance(S)
            
            vol_ = sum(g.d[S])
            true_positives_l1SL_ = set(rr).intersection(S)
            if len(true_positives_l1SL_) == 0:
                true_positives_l1SL_ = set(ref_node)
                vol_ = g.d[ref_node][0]
            precision = sum(g.d[np.array(list(true_positives_l1SL_))])/vol_
            recall = sum(g.d[np.array(list(true_positives_l1SL_))])/sum(g.d[rr])
            f1_score_ = 2*(precision*recall)/(precision + recall)
            
            if f1_score_ >= max_precision:
                
                max_precision = f1_score_
                
                external_best_pre_cond_l1SL[ct_outer,node] = cond_val_l1pr
                vol_best_pre_l1SL[ct_outer,node] = vol_
                
                size_clust_best_pre_l1SL[ct_outer,node] = size_clust_l1SL_
                true_positives_best_pre_l1SL[ct_outer,node] = true_positives_l1SL_
                precision_best_pre_l1SL[ct_outer,node] = precision
                recall_best_pre_l1SL[ct_outer,node] = recall
                f1score_best_pre_l1SL[ct_outer,node] = f1_score_
                
                cuts_best_pre_l1SL[ct_outer,node] = S
        
            if cond_val_l1pr <= min_conduct:
                
                min_conduct = cond_val_l1pr
                
                external_best_cond_l1SL[ct_outer,node] = cond_val_l1pr
                vol_best_cond_l1SL[ct_outer,node] = vol_
                
                size_clust_best_cond_l1SL[ct_outer,node] = size_clust_l1SL_
                true_positives_best_cond_l1SL[ct_outer,node] = true_positives_l1SL_
                precision_best_cond_l1SL[ct_outer,node] = precision
                recall_best_cond_l1SL[ct_outer,node] = recall
                f1score_best_cond_l1SL[ct_outer,node] = f1_score_
                
                cuts_best_cond_l1SL[ct_outer,node] = S

        print('outer:', ct_outer, 'number of node: ',node, ' completed: ', ct/how_many, ' degree: ', g.d[node])
        print('conductance: ', external_best_cond_l1SL[ct_outer,node], 'f1score: ', f1score_best_pre_l1SL[ct_outer,node], 'precision: ', precision_best_pre_l1SL[ct_outer,node], 'recall: ', recall_best_pre_l1SL[ct_outer,node])
        ct += 1
    end = time.time()
    print(" ")
    print("Outer: ", ct_outer," Elapsed time L1+SL with rounding: ", end - start)
    print("Outer: ", ct_outer," Number of experiments: ", number_experiments)
    print(" ")
    ct_outer += 1

24
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.7158852712756061
outer: 0 number of node:  843  completed:  0.0  degree:  41.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  858  completed:  0.041666666666666664  degree:  25.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.39964594813472987
outer: 0 number of node:  839  completed:  0.08333333333333333  degree:  45.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  850  completed:  0.125  degree:  34.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.24062924531923566
outer: 0 number of node:  836  comple

/usr/local/Cellar/ipython/7.1.1/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning:

divide by zero encountered in double_scalars



outer: 0 number of node:  842  completed:  0.375  degree:  42.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  848  completed:  0.4166666666666667  degree:  36.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  845  completed:  0.4583333333333333  degree:  39.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  846  completed:  0.5  degree:  38.0
conductance:  0.3370165745856354 f1score:  0.9802816901408451 precision:  0.9613259668508287 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 0 number of node:  849  completed:  0.5416666666666666  degree:  35.0
conductance:  0.3370165745856354 f1score:  0.98028169014084

DELTA:  0.013832143356004684
outer: 3 number of node:  609  completed:  0.2631578947368421  degree:  28.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
DELTA:  1
DELTA:  0.11490534424181181
DELTA:  0.024498195129787924
DELTA:  0.024498195129787924
outer: 3 number of node:  590  completed:  0.3157894736842105  degree:  48.0
conductance:  0.3603707995365006 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  0.16045300161858386
DELTA:  0.012803002488475502
outer: 3 number of node:  611  completed:  0.3684210526315789  degree:  26.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7833140208574739 recall:  1.0
DELTA:  1
DELTA:  0.500167817316623
DELTA:  0.031242842722267933
DELTA:  0.018015545944181487
outer: 3 number of node:  601  completed:  0.42105263157894735  degree:  36.0
conductance:  0.3582887700534759 f1score:  0.8784925276153347 precision:  0.7

outer: 4 number of node:  75  completed:  0.2716049382716049  degree:  147.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.0940211437714115
outer: 4 number of node:  74  completed:  0.2839506172839506  degree:  148.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.2220179644784689
outer: 4 number of node:  89  completed:  0.2962962962962963  degree:  133.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 4 number of node:  113  completed:  0.30864197530864196  degree:  109.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 4 number of node:  109  completed:  0.32098765432098764  degree:  113.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA: 

...

DELTA:  1
outer: 8 number of node:  468  completed:  0.875  degree:  46.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.8452286198543848
outer: 8 number of node:  467  completed:  0.90625  degree:  47.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 8 number of node:  469  completed:  0.9375  degree:  45.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 8 number of node:  472  completed:  0.96875  degree:  42.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
 
Outer:  8  Elapsed time L1+SL with rounding:  18.66217303276062
Outer:  8  Number of experiments:  1056
 
14
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 9 number of node:  879  completed:  0.0  degree:  13.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0

DELTA:  0.009880251953791545
outer: 11 number of node:  353  completed:  0.15151515151515152  degree:  60.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.05338043756218871
outer: 11 number of node:  371  completed:  0.18181818181818182  degree:  42.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELTA:  1
DELTA:  0.6311455005820304
DELTA:  0.04083463579088538
DELTA:  0.011945328068791153
outer: 11 number of node:  341  completed:  0.21212121212121213  degree:  72.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  0.0933314625932441
DELTA:  0.01073696294044286
outer: 11 number of node:  348  completed:  0.24242424242424243  degree:  65.0
conductance:  0.3333333333333333 f1score:  0.9375314861460956 precision:  0.8824087245139877 recall:  1.0
DELT

DELTA:  1
DELTA:  0.09904435808279942
DELTA:  0.024224770086131053
DELTA:  0.024224770086131053
outer: 12 number of node:  558  completed:  0.5555555555555556  degree:  43.0
conductance:  0.3619167717528373 f1score:  0.8727789623312011 precision:  0.7742749054224464 recall:  1.0
DELTA:  1
DELTA:  0.37202498769098663
DELTA:  0.020026791618921975
DELTA:  0.020026791618921975
outer: 12 number of node:  565  completed:  0.6111111111111112  degree:  36.0
conductance:  0.3619167717528373 f1score:  0.8727789623312011 precision:  0.7742749054224464 recall:  1.0
DELTA:  1
DELTA:  0.37449772438369816
DELTA:  0.020620056829536076
DELTA:  0.020620056829536076
outer: 12 number of node:  564  completed:  0.6666666666666666  degree:  37.0
conductance:  0.3619167717528373 f1score:  0.8727789623312011 precision:  0.7742749054224464 recall:  1.0
DELTA:  1
DELTA:  0.3011511158521298
DELTA:  0.02121545204771426
DELTA:  0.02121545204771426
outer: 12 number of node:  563  completed:  0.7222222222222222  deg

DELTA:  0.01035842932458952
outer: 13 number of node:  393  completed:  0.8275862068965517  degree:  49.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.6689158794421952 recall:  1.0
DELTA:  1
DELTA:  0.331810909050364
DELTA:  0.021752868171911764
DELTA:  0.013909873100751176
outer: 13 number of node:  380  completed:  0.8620689655172413  degree:  65.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.6689158794421952 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  0.06936214314009394
DELTA:  0.011458974912373054
outer: 13 number of node:  389  completed:  0.896551724137931  degree:  54.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.6689158794421952 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  0.23999271508527384
DELTA:  0.009266197561986003
outer: 13 number of node:  398  completed:  0.9310344827586207  degree:  44.0
conductance:  0.34053081421502474 f1score:  0.8016172506738545 precision:  0.668915879442195

DELTA:  1
DELTA:  0.5473042018637221
outer: 15 number of node:  281  completed:  0.7  degree:  15.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 15 number of node:  284  completed:  0.8  degree:  12.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 15 number of node:  285  completed:  0.9  degree:  11.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
 
Outer:  15  Elapsed time L1+SL with rounding:  24.88539695739746
Outer:  15  Number of experiments:  1608
 
11
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  0.428656006953664
outer: 16 number of node:  719  completed:  0.0  degree:  18.0
conductance:  0.3333333333333333 f1score:  1.0 precision:  1.0 recall:  1.0
DELTA:  1
DELTA:  1
DELTA:  1
DELTA:  1
outer: 16 number of node:  723  completed:  0.09090909090909091  degree:  14.0
conductance:  0.3333333333333333 f1scor

## Performance of l1+SL

In [34]:
all_data = []
xlabels_ = []

print('Results for L1+SL')
sum_precision = 0
sum_recall = 0
sum_f1 = 0
sum_conductance = 0

info_ref_nodes = all_clusters
l_info_ref_nodes = len(info_ref_nodes)

for i in range(l_info_ref_nodes):
    temp_pre = []
    temp_rec = []
    temp_f1 = []
    temp_conductance = []
    
    for j in all_clusters[i]:
        temp_pre.append(precision_best_cond_l1SL[i,j])
        temp_rec.append(recall_best_cond_l1SL[i,j])
        temp_f1.append(f1score_best_cond_l1SL[i,j])
        temp_conductance.append(external_best_cond_l1SL[i,j])

    print('Feature:', i,'Precision', stat_.mean(temp_pre), 'Recall', stat_.mean(temp_rec), 'F1', stat_.mean(temp_f1), 'Cond.', stat_.mean(temp_conductance))
    

Results for L1+SL
Feature: 0 Precision 0.9613259668508287 Recall 1.0 F1 0.9802816901408451 Cond. 0.3370165745856354
Feature: 1 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 2 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 3 Precision 0.7483922812279147 Recall 1.0 F1 0.8557617594794483 Cond. 0.3591654140463284
Feature: 4 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 5 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 6 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 7 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 8 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 9 Precision 1.0 Recall 1.0 F1 1.0 Cond. 0.3333333333333333
Feature: 10 Precision 0.6862962743727526 Recall 0.958185053380783 F1 0.7929438950135047 Cond. 0.4206780727399051
Feature: 11 Precision 0.8824087245139877 Recall 1.0 F1 0.9375314861460956 Cond. 0.3333333333333333
Feature: 12 Precision 0.742050